In [1]:
!pip install nbimporter

In [28]:
!pip install ipynb

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from pathlib import Path
import pickle
import nbimporter
import torch.nn.functional as F

In [27]:
# Import architectures
from recommenders_architecture import *

### Load data

In [28]:
# ======= Load Pairwise Training Data =======
current_dir = Path.cwd()

pairwise_data_train_path= current_dir.parent / "data" / "pairwise"/"pairwise_train.csv"
train = pd.read_csv(pairwise_data_train_path)
pairwise_data_val_path= current_dir.parent / "data" / "pairwise"/"pairwise_val.csv"
val = pd.read_csv(pairwise_data_val_path)


# ======= Load Item Metadata (1027-dim vectors) =======

encoded_dir = current_dir.parent / "data" / "encoded"
encoded_text_file = encoded_dir / "embedding_dict_with_price_longformer_idx.pt"
encoded_images_file = encoded_dir / "images_encodings.pkl"
encoded_metadata_text_image_file = encoded_dir / "item_metadata_text_image.pt"
auto_encoder_metadata_file = encoded_dir / "compressed_all_data_encodings_256.pkl"


text_embeddings = torch.load(encoded_text_file)
# with open(encoded_images_file, 'rb') as f:
#     images_embeddings = pickle.load(f)


In [29]:
train

,user_id,item1_id,item2_id,label,timestamp,rating
0,0,0,13349,0,1349041740000,5.0
1,0,22959,1,1,1370958618000,1.0
2,0,97562,2,2,1440038761000,5.0
3,0,23003,3,3,1483320893000,3.0
4,1,179127,5,5,1600753653091,5.0
...,...,...,...,...,...,...
8030471,1096899,26803,32852,32852,1692552324736,5.0
8030472,1096899,177842,10643,10643,1692552357767,5.0
8030473,1096900,183765,86867,183765,1600792118191,1.0
8030474,1096900,155119,99585,155119,1615811081145,1.0


In [30]:

# ======= Configurations =======
EMBEDDING_DIM = 24  # User embedding size
ITEM_FEATURE_DIM = 1027# item_metadata[0].shape # Length of item metadata vector (text+image)
#ITEM_FEATURE_DIM = 256 # After autoencoder
BATCH_SIZE = 512
EPOCHS = 10
LR = 0.00001  # Learning rate
VAL_SPLIT = 0.1

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### if we choose to use Auto encoder data instead of the long text embeddings

In [13]:
# with open(auto_encoder_metadata_file, 'rb') as f:
#     item_metadata = pickle.load(f)

In [92]:

# item_embeddings_tensor = torch.zeros(len(item_metadata), item_metadata[0].size(0))
# for idx, (item_id, embed) in enumerate(item_metadata.items()):
#     item_embeddings_tensor[idx] = embed


In [93]:
# auto_encoder_metadata_file_pt_out = encoded_dir / "compressed_all_data_encodings_256.pt"

In [94]:
# torch.save(item_embeddings_tensor,auto_encoder_metadata_file_pt_out )

### Load pretrained embeddings from yahlly's model

In [33]:
user_item_file_path = current_dir.parent / "data" / "data_and_test_files" / "user_item_rating_table_train_with_idx.csv"
df2 = pd.read_csv(user_item_file_path)

### Concat image and text embeddings

##### Create concat of image and text

In [47]:
# import torch

# # ======= Create Combined Item Metadata =======
# item_metadata = {}

# for item_id in text_embeddings.keys():
#     text_embed = text_embeddings[item_id]  # (1027,)
#     image_embed = images_embeddings.get(item_id, torch.zeros(2048))  # (2048,) default to zeros if missing

#     # Concatenate along the feature dimension
#     combined_embed = torch.cat([text_embed, image_embed], dim=0)  # (3075,)
#     item_metadata[item_id] = combined_embed

# # Save the combined metadata dictionary
# metadata_save_path = encoded_dir / "item_metadata_text_image.pt"
# torch.save(item_metadata, metadata_save_path)

# print(f"✅ Item metadata saved to {metadata_save_path} with {len(item_metadata)} items.")


✅ Item metadata saved to /storage/yahlly/RecSys/data/encoded/item_metadata_text_image.pt with 198771 items.


### Load item_metadata (text+image embeddings)

In [5]:
item_metadata = torch.load( encoded_dir / "item_metadata_text_image.pt")

In [34]:
item_metadata = text_embeddings

#### make the item metadata a tensor

In [35]:
item_embeddings_tensor = torch.zeros(len(item_metadata), item_metadata[0].size(0))
for idx, (item_id, embed) in enumerate(item_metadata.items()):
    item_embeddings_tensor[idx] = embed


In [36]:
item_embeddings_tensor=item_embeddings_tensor.to(device)

In [13]:
df2

,user_id,parent_asin,rating,timestamp,user_idx,item_idx
0,AGGZ357AO26RQZVRLGU4D4N52DZQ,B009RTBRVG,5.0,1349041740000,0,0
1,AGGZ357AO26RQZVRLGU4D4N52DZQ,B003MZ01CM,1.0,1370958618000,0,1
2,AGGZ357AO26RQZVRLGU4D4N52DZQ,B07L6QT33F,5.0,1440038761000,0,2
3,AGGZ357AO26RQZVRLGU4D4N52DZQ,B07V6PKCCG,3.0,1483320893000,0,3
4,AGGZ357AO26RQZVRLGU4D4N52DZQ,B0BNP511CS,5.0,1490800837000,0,4
...,...,...,...,...,...,...
9127372,AEQHNMSCENA2TJAJEFK5SFI3ZKXA,B09G9THPC6,5.0,1692552496934,1096899,92761
9127373,AFGBVYKTFNQH5NIHXNB5ANVPANXQ,B088D217BQ,1.0,1600792118191,1096900,183765
9127374,AFGBVYKTFNQH5NIHXNB5ANVPANXQ,B07WV5H4DN,1.0,1615811081145,1096900,155119
9127375,AFGBVYKTFNQH5NIHXNB5ANVPANXQ,B0BL3PQHR4,4.0,1693494834857,1096900,25515


### Data loader class

In [37]:
# ======= Custom Dataset Class =======
class PairwiseDataset(Dataset):
    def __init__(self, dataframe):
        self.users = dataframe["user_id"].values
        self.item1 = dataframe["item1_id"].values
        self.item2 = dataframe["item2_id"].values
        self.labels = dataframe["label"].values

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return (
            self.users[idx],
            self.item1[idx],
            self.item2[idx],
            self.labels[idx],
        )


In [43]:
train

,user_id,item1_id,item2_id,label,timestamp,rating
0,0,0,13349,0,1349041740000,5.0
1,0,22959,1,1,1370958618000,1.0
2,0,97562,2,2,1440038761000,5.0
3,0,23003,3,3,1483320893000,3.0
4,1,179127,5,5,1600753653091,5.0
...,...,...,...,...,...,...
8030471,1096899,26803,32852,32852,1692552324736,5.0
8030472,1096899,177842,10643,10643,1692552357767,5.0
8030473,1096900,183765,86867,183765,1600792118191,1.0
8030474,1096900,155119,99585,155119,1615811081145,1.0


In [38]:
import torch

# Map item embeddings for fast lookup
item_embeddings_dict = {idx: item_embeddings_tensor[idx] for idx in range(len(item_embeddings_tensor))}

# Compute user embeddings
user_embeddings = torch.zeros((df2["user_id"].nunique(), item_embeddings_tensor.shape[1]))

for user_id, group in tqdm(train.groupby("user_id")):
    # Get item embeddings for this user's interactions
    item_embeddings = torch.stack([item_embeddings_dict[i] for i in group["label"].values])
    # Center of mass: mean of all interacted item embeddings
    user_embeddings[user_id] = item_embeddings.mean(dim=0)

# Move user embeddings to the appropriate device
#user_embeddings = user_embeddings.to(device)


 11%|█         | 117829/1096358 [00:15<02:07, 7680.45it/s]


KeyboardInterrupt: 

In [33]:
# Save the embeddings with a fixed filename
torch.save(user_embeddings, encoded_dir / "user_embeddings_using_mean_on_metadata_text_only.pt")

In [40]:
user_embeddings_path = encoded_dir / "user_embeddings_using_mean_on_metadata_text_only.pt"

# Load the embeddings
user_embeddings = torch.load(user_embeddings_path)

### Saving as V (not mean)

In [9]:
# import torch
# from tqdm import tqdm

# # Map item embeddings for fast lookup
# item_embeddings_dict = {idx: item_embeddings_tensor[idx] for idx in range(len(item_embeddings_tensor))}

# # Store all item embeddings for each user
# user_item_embeddings = {}

# for user_id, group in tqdm(train.groupby("user_id")):
#     # Get item embeddings for this user's interactions
#     item_embeddings = torch.stack([item_embeddings_dict[i] for i in group["label"].values])
    
#     # Store the item embeddings for this user
#     user_item_embeddings[user_id] = item_embeddings
# # 
# # Example: Access item embeddings for user 0
# print("User 0 embeddings shape:", user_item_embeddings[0].shape)


100%|██████████| 1096358/1096358 [01:19<00:00, 13823.18it/s]

User 0 embeddings shape: torch.Size([4, 1027])


In [11]:
torch.save(user_item_embeddings, encoded_dir / "user_embeddings_using_metadata_text_only_all_interacted_items.pt")

In [10]:
user_item_embeddings.shape

AttributeError: 'dict' object has no attribute 'shape'

In [12]:
user_embeddings = user_item_embeddings

In [41]:
user_embeddings = user_embeddings.to(device)

In [15]:
user_embeddings.shape

torch.Size([1096901, 3075])

In [28]:
user_embeddings.shape

torch.Size([1096901, 3075])

In [42]:
# ======= Two-Tower Model (User & Item Networks) =======
class TwoTowerModel(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim, item_metadata_dim):
        super(TwoTowerModel, self).__init__()
        
        # User Tower (Embedding)
        self.user_fc = nn.Sequential(
            nn.Linear(1027, 512),  # First reduce to 512 dimensions
            nn.ReLU(),
            nn.Linear(512, embedding_dim),  # Then reduce to the desired embedding_dim (24)
        )
        # Item Tower (Using Item Metadata)
        self.item_fc = nn.Sequential(
            nn.Linear(item_metadata_dim, 512),
            nn.ReLU(),
            nn.Linear(512, embedding_dim),
        )

        
    def forward(self, user_ids, item1_ids, item2_ids):
        user_ids = user_ids.long().to(device)
        
        item1_ids=item1_ids.to(device)
        item2_ids=item2_ids.to(device)
        # User Tower: Compute user embeddings using user_fc (sequential)
        user_embed = self.user_fc(user_embeddings[user_ids])  # (batch_size, embedding_dim)

        item1_embed = self.item_fc(item_embeddings_tensor[item1_ids])  # (batch, embedding_dim)
        item2_embed = self.item_fc(item_embeddings_tensor[item2_ids])  # (batch, embedding_dim)
        
        return user_embed, item1_embed, item2_embed

# ======= Pairwise BPR Loss =======


class BPRLoss(nn.Module):
    def __init__(self):
        super(BPRLoss, self).__init__()

    def forward(self, user_embed, item1_ids, item1_embed, item2_ids, item2_embed, labels):
        """
        Compute Bayesian Personalized Ranking (BPR) loss.

        Args:
        - user_embed: Tensor of shape (batch_size, embed_dim), user embeddings.
        - item1_ids: Tensor of shape (batch_size,), IDs of item1.
        - item1_embed: Tensor of shape (batch_size, embed_dim), embeddings for item1.
        - item2_ids: Tensor of shape (batch_size,), IDs of item2.
        - item2_embed: Tensor of shape (batch_size, embed_dim), embeddings for item2.
        - labels: Tensor of shape (batch_size,), IDs of the correct (positive) item.

        Returns:
        - loss: Computed BPR loss.
        """
        # Convert labels to binary: 1 if item1 is the positive item, else 0
        labels_binary = (labels == item1_ids).float()

        # Compute scores
        score1 = (user_embed * item1_embed).sum(dim=1)  # Affinity score for item1
        score2 = (user_embed * item2_embed).sum(dim=1)  # Affinity score for item2

        # Assign correct positive and negative scores based on labels_binary
        pos_score = torch.where(labels_binary == 1, score1, score2)
        neg_score = torch.where(labels_binary == 1, score2, score1)

        # Compute BPR loss
        loss = -F.logsigmoid(pos_score - neg_score).mean()
        return loss


In [20]:
df.head()

,user_id,item1_id,item2_id,label
0,0,50787,0,0
1,0,1,33482,1
2,0,35983,2,2
3,0,3,125581,3
4,0,4,60296,4


In [43]:
train_loader = DataLoader(PairwiseDataset(train), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(PairwiseDataset(val), batch_size=BATCH_SIZE, shuffle=False)

In [44]:
num_users = 1096901
num_items = 198771


In [45]:

# ======= Initialize Model, Loss, Optimizer =======
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TwoTowerModel(num_users, num_items, EMBEDDING_DIM, ITEM_FEATURE_DIM).to(device)
criterion = BPRLoss()
optimizer = optim.Adam(model.parameters(), lr=LR*0.1)

# ======= Training Loop =======
# ======= Training & Validation =======
log_file = "cold_training_log.txt"

print("🚀 Training Model...")
with open(log_file, "w") as log:
    log.write("🚀 Training Model...\n")
    for epoch in range(EPOCHS):
        model.train()
        train_loss = 0
    
        for user_ids, item1_ids, item2_ids,labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
            user_ids, item1_ids, item2_ids,labels = user_ids.to(device), item1_ids.to(device), item2_ids.to(device), labels.to(device)
    
            # Forward Pass
            user_embed, item1_embed, item2_embed = model(user_ids, item1_ids, item2_ids)
            #print(item1_embed==item2_embed)
            # Compute Loss
            # print(item1_embed==item2_embed)
            loss = criterion(user_embed,item1_ids, item1_embed,item2_ids, item2_embed, labels)
            train_loss += loss.item()
            
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_loss = train_loss /len(train_loader)
        # ======= Validation =======
        model.eval()
        correct = 0
        total = 0
        val_loss = 0
        for user_ids, item1_ids, item2_ids, labels in val_loader:
            user_ids, item1_ids, item2_ids, labels = (
                user_ids.to(device),
                item1_ids.to(device),
                item2_ids.to(device),
                labels.to(device),
            )
            user_embed, item1_embed, item2_embed = model(user_ids, item1_ids, item2_ids)
            #print((item1_embed==item2_embed).all())
            score1 = (user_embed * item1_embed).sum(dim=1)  # Score for item1
            score2 = (user_embed * item2_embed).sum(dim=1)  # Score for item2
    
            # Determine the correct positive and negative scores based on labels
            labels_binary = (labels == item1_ids).float()
            #print(labels_binary)
            pos_scores = torch.where(labels_binary == 1, score1, score2)
            neg_scores = torch.where(labels_binary == 1, score2, score1)
            #print(pos_scores)
            # Check if the model correctly ranked the positive item higher
            loss = criterion(user_embed,item1_ids, item1_embed,item2_ids, item2_embed, labels)
            val_loss += loss.item()
            predictions = pos_scores > neg_scores
    
            correct += predictions.sum().item()
            total += predictions.shape[0]
    
        val_accuracy = correct / total
        val_loss=val_loss/len(val_loader)
        print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f},Val Loss = {val_loss:.4f}, Val Accuracy = {val_accuracy:.4f}")
        log.write(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Val Accuracy = {val_accuracy:.4f}\n")



# ======= Save Model =======
#torch.save(model.state_dict(), "trained_model.pth")
print("✅ Model Training Complete!")
with open(log_file, "a") as log:
    log.write("✅ Model Training Complete!\n")

🚀 Training Model...


Epoch 1/10: 100%|██████████| 15685/15685 [01:15<00:00, 208.65it/s]


Epoch 1: Train Loss = 0.6755,Val Loss = 0.7453, Val Accuracy = 0.5000


Epoch 2/10: 100%|██████████| 15685/15685 [01:14<00:00, 209.42it/s]


Epoch 2: Train Loss = 0.6502,Val Loss = 0.7547, Val Accuracy = 0.5006


...

Epoch 8: Train Loss = 0.5897,Val Loss = 0.8596, Val Accuracy = 0.5027


Epoch 9/10: 100%|██████████| 15685/15685 [01:14<00:00, 209.19it/s]


Epoch 9: Train Loss = 0.5875,Val Loss = 0.8600, Val Accuracy = 0.5029


Epoch 10/10: 100%|██████████| 15685/15685 [01:15<00:00, 209.08it/s]


Epoch 10: Train Loss = 0.5829,Val Loss = 0.8607, Val Accuracy = 0.5030
✅ Model Training Complete!


##### PREDICT ON COLD ITEMS

In [46]:
pairwise_data_test_path= current_dir.parent / "data" / "data_and_test_files"/"cold_items_classification_test_format.csv" 
test = pd.read_csv(pairwise_data_test_path)
test

,user_id,item_0,item_1,class
0,AEWJXW46LKSW22FWTGRYTLNE7TGA,B0796QVRYW,B098F8FLYJ,-1
1,AEWLOQTKYUYRJ24NK4GD5CZTR7WA,B0BJ34VXNQ,B01MFBK6J1,-1
2,AH3OG6QD6EDJGZRVCFKV4B66VWNQ,B096M84Y7M,B002YQ2GIW,-1
3,AGVVUU3QRQBHNASSGI5YQLPYOI2Q,B0C1N7N7V7,B09DVKV5Z6,-1
4,AHY55PORR7VYL72JBUYNSO6KCA5A,B00UXT39KG,B0C2DGBGJJ,-1
...,...,...,...,...
67583,AFIDVM6FUBIYOBJONKOPATBBBUBQ,B00846P0Q6,B09YMRKZ7S,-1
67584,AH6RGQADHHZBIDZJAI7BVFZPNOHA,B01EIPIHYI,B08FCLJV83,-1
67585,AFZ5UTEAQL7GGI6ISFZZAN2ERO5Q,B0072J71KO,B0C3KV8LH4,-1
67586,AGA6K6MEKC3KPDC2WWBT4Y5OUR6Q,B08K7BJM9R,B01HNQ1OX0,-1


In [49]:
user_mapping_path = current_dir.parent / "data" / "pre_process" / "user_mapping.pkl"
item_mapping_path = current_dir.parent / "data" / "pre_process" / "item_mapping.pkl"
import pickle
# Load user and item mappings
with open(user_mapping_path, 'rb') as f:
    user_mapping = pickle.load(f)

with open(item_mapping_path, 'rb') as f:
    item_mapping = pickle.load(f)

# Add numeric indices to the DataFrame
test['user_idx'] = test['user_id'].map(user_mapping)
test['item1_id'] = test['item_0'].map(item_mapping)
test['item2_id'] = test['item_1'].map(item_mapping)
test

,user_id,item_0,item_1,class,user_idx,item1_id,item2_id
0,AEWJXW46LKSW22FWTGRYTLNE7TGA,B0796QVRYW,B098F8FLYJ,-1,14,NaN,NaN
1,AEWLOQTKYUYRJ24NK4GD5CZTR7WA,B0BJ34VXNQ,B01MFBK6J1,-1,57,NaN,NaN
2,AH3OG6QD6EDJGZRVCFKV4B66VWNQ,B096M84Y7M,B002YQ2GIW,-1,63,NaN,NaN
3,AGVVUU3QRQBHNASSGI5YQLPYOI2Q,B0C1N7N7V7,B09DVKV5Z6,-1,84,NaN,NaN
4,AHY55PORR7VYL72JBUYNSO6KCA5A,B00UXT39KG,B0C2DGBGJJ,-1,87,NaN,NaN
...,...,...,...,...,...,...,...
67583,AFIDVM6FUBIYOBJONKOPATBBBUBQ,B00846P0Q6,B09YMRKZ7S,-1,1096816,NaN,NaN
67584,AH6RGQADHHZBIDZJAI7BVFZPNOHA,B01EIPIHYI,B08FCLJV83,-1,1096833,NaN,NaN
67585,AFZ5UTEAQL7GGI6ISFZZAN2ERO5Q,B0072J71KO,B0C3KV8LH4,-1,1096847,NaN,NaN
67586,AGA6K6MEKC3KPDC2WWBT4Y5OUR6Q,B08K7BJM9R,B01HNQ1OX0,-1,1096879,NaN,NaN


In [47]:
val

,user_id,item1_id,item2_id,label,timestamp,rating
0,0,16177,4,4,1490800837000,5.0
1,1,10,174536,10,1676601720832,2.0
2,2,42860,16,16,1588626339041,5.0
3,3,20877,29,29,1605455790941,5.0
4,4,17870,41,41,1638039645551,5.0
...,...,...,...,...,...,...
1096896,1096896,197,11404,11404,1693892929945,5.0
1096897,1096897,32215,161020,161020,1617640776113,5.0
1096898,1096898,9974,33337,9974,1691348903005,5.0
1096899,1096899,45300,92761,92761,1692552496934,5.0


In [ ]:
# ======= Custom Dataset Class =======
class PairwiseDataset(Dataset):
    def __init__(self, dataframe):
        self.users = dataframe["user_id"].values
        self.item1 = dataframe["item1_id"].values
        self.item2 = dataframe["item2_id"].values
        self.labels = dataframe["label"].values

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return (
            self.users[idx],
            self.item1[idx],
            self.item2[idx],
            self.labels[idx],
        )

In [ ]:
model.eval()
correct = 0
total = 0
val_loss = 0
for user_ids, item1_ids, item2_ids, labels in val_loader:
    user_ids, item1_ids, item2_ids, labels = (
        user_ids.to(device),
        item1_ids.to(device),
        item2_ids.to(device),
        labels.to(device),
    )
    user_embed, item1_embed, item2_embed = model(user_ids, item1_ids, item2_ids)
    #print((item1_embed==item2_embed).all())
    score1 = (user_embed * item1_embed).sum(dim=1)  # Score for item1
    score2 = (user_embed * item2_embed).sum(dim=1)  # Score for item2

    # Determine the correct positive and negative scores based on labels
    labels_binary = (labels == item1_ids).float()
    #print(labels_binary)
    pos_scores = torch.where(labels_binary == 1, score1, score2)
    neg_scores = torch.where(labels_binary == 1, score2, score1)
    #print(pos_scores)
    # Check if the model correctly ranked the positive item higher
    loss = criterion(user_embed,item1_ids, item1_embed,item2_ids, item2_embed, labels)
    val_loss += loss.item()
    predictions = pos_scores > neg_scores

    correct += predictions.sum().item()
    total += predictions.shape[0]

val_accuracy = correct / total
val_loss=val_loss/len(val_loader)
print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f},Val Loss = {val_loss:.4f}, Val Accuracy = {val_accuracy:.4f}")
log.write(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}, Val Accuracy = {val_accuracy:.4f}\n")
